In [1]:
#读取h5 文件
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import h5py
import sys
from numba import jit
import seaborn as sns
from mylab.ana.miniscope.Mquality import *
from mylab.ana.miniscope.Mgraph import *
import scipy.io as scio

C:\Users\Sabri\Anaconda3\envs\mylab\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)


In [2]:
f = scio.loadmat(r"aCA3_pre_post_all_cells_excl_freeze.mat")
keys = [i for i in f.keys() if r"__" not in i]
keys

['ShockActivated',
 'ShockInhibited',
 'ShockNoResponse',
 'postA',
 'postB',
 'preA',
 'preB']

In [3]:
ShockActivated = pd.DataFrame([(int(np.floor(i/10000)),i%10000) for i in list(f['ShockActivated'].ravel())],columns=["mouse_id","cell_id"])
ShockActivated["type"] = "SA"
ShockInhibited = pd.DataFrame([(int(np.floor(i/10000)),i%10000) for i in list(f['ShockInhibited'].ravel())],columns=["mouse_id","cell_id"])
ShockInhibited["type"] = "SI"
ShockNoResponse =pd.DataFrame([(int(np.floor(i/10000)),i%10000) for i in  list(f['ShockNoResponse'].ravel())],columns=["mouse_id","cell_id"])
ShockNoResponse["type"] = "SN"

In [4]:
# neuronal infomation
all_neuon_type = pd.concat([ShockActivated,ShockInhibited,ShockNoResponse])
all_neuon_type=all_neuon_type.sort_values(by=["mouse_id","cell_id"],ignore_index=True)
all_neuon_type

,mouse_id,cell_id,type
0,1,1,SA
1,1,2,SN
2,1,3,SI
3,1,4,SN
4,1,5,SI
...,...,...,...
713,8,95,SN
714,8,96,SI
715,8,97,SA
716,8,98,SN


In [140]:
# sessions
preA = np.array(f["preA"]).T
preB = np.array(f["preB"]).T
postA = np.array(f["postA"]).T
postB = np.array(f["postB"]).T
sessions = ["preA","preB","postA","postB"]

In [ ]:
# defined shock-partner cells

In [13]:
for mouseid in [1,2,3,4,5,6,7,8]:
    
    for session in ["preA","preB","postA","postB"]:
        # firing in session
        arr = f[session].T[all_neuon_type["mouse_id"]==mouseid]  
        #df in session
        df = all_neuon_type[all_neuon_type["mouse_id"]==mouseid].reset_index(drop=True)
        
        cell_ids = df["cell_id"].tolist()
        
#         cpr = compute_correlated_pair_ratio(arr,cell_ids)
#         cc = comput_clustering_coefficient_by_session(arr,cell_ids)
        
        cp = comput_component_probability_by_session(arr,cell_ids,threshold=0.8)
        print(mouseid,session,cp)
        
        
    

1 preA 1.0
1 preB 1.0
1 postA 1.0
1 postB 1.0
2 preA 1.0
2 preB 1.0
2 postA 1.0
2 postB 1.0
3 preA 1.0
3 preB 1.0
3 postA 1.0
3 postB 1.0
4 preA 1.0
4 preB 1.0
4 postA 1.0
4 postB 1.0
5 preA 1.0
5 preB 1.0
5 postA 1.0
5 postB 1.0
6 preA 1.0
6 preB 1.0
6 postA 1.0
6 postB 1.0
7 preA 1.0
7 preB 1.0
7 postA 1.0
7 postB 1.0
8 preA 1.0
8 preB 1.0
8 postA 1.0
8 postB 1.0


# definition

### correlated pair ratio
**Significantly correlated pairs of neurons** were defined based on
the correlation coefficients of Ca2+ transient events within one
second time bins during context exploration.

**were computed for each neuron**

$$The\ correlated\ pair\ ratio =\frac{\#pairs}{\#total\ cells\ in\ FOV}$$


`An R thresholds of 0.3 was applied to this matrix to reduce bias from noise.
R thresholds at p < 0.05 from the shuffle distribution were then applied to determine the number of correlated pairs with shuffled R thresholds(Jimenez et al. 2020)`


            
        


### clustering coefficient
 the
extent to which a given neuron’s pairs were also correlated with
each other (clustering coefficient)
$$Clustering\ coefficient =\frac{\# edges\ among\ neighbors}{\# possible\ edges\ among\ neighbors} $$

`Connection density is the actual number of edges in the graph as a proportion of the total number of possible edges and is the simplest estimator of the physical cost — for example, the energy or other resource requirements — of a network. (Bullmore et al. 2009)`

**were computed for each neuron**

Total number of possible edges is: n(n-1)/2,where n is the number of nodes (neurons) in the graph.

correlated function in class NeuronalNetwork: `self.compute_connection_density()`


### component probability
A connected component is defined as a subgraph of nodes connected to each other by paths but not
connected to any other nodes. **Each neuron was assigned either a 0 or 1 indicating
whether it was a component member**, and the proportion of neurons in a component (component probability) was computed for each imaging session



correlated function in module `Mgraph.py`
`frac_nodes_in_component(g,threshold)`
**threshold** means the minimum component size.
`Due to the sparse nature of these neural graphs, we also determined whether each neuron was as member of a connected component containing at least two other neurons(Jimenez et al. 2020)`which means threshold in this paper is **3**.
$$Component\ probability =\frac{\#\ neurons\  in\  a\ component}{\#neurons\ in\  session}$$